In [1]:
%matplotlib inline


Automatic Mixed Precision
*************************
**Author**: `Michael Carilli <https://github.com/mcarilli>`_

`torch.cuda.amp <https://pytorch.org/docs/stable/amp.html>`_ provides convenience methods for mixed precision,
where some operations use the ``torch.float32`` (``float``) datatype and other operations
use ``torch.float16`` (``half``). Some ops, like linear layers and convolutions,
are much faster in ``float16``. Other ops, like reductions, often require the dynamic
range of ``float32``.  Mixed precision tries to match each op to its appropriate datatype,
which can reduce your network's runtime and memory footprint.

Ordinarily, "automatic mixed precision training" uses `torch.cuda.amp.autocast <https://pytorch.org/docs/stable/amp.html#torch.cuda.amp.autocast>`_ and
`torch.cuda.amp.GradScaler <https://pytorch.org/docs/stable/amp.html#torch.cuda.amp.GradScaler>`_ together.

This recipe measures the performance of a simple network in default precision,
then walks through adding ``autocast`` and ``GradScaler`` to run the same network in
mixed precision with improved performance.

You may download and run this recipe as a standalone Python script.
The only requirements are Pytorch 1.6+ and a CUDA-capable GPU.

Mixed precision primarily benefits Tensor Core-enabled architectures (Volta, Turing, Ampere).
This recipe should show significant (2-3X) speedup on those architectures.
On earlier architectures (Kepler, Maxwell, Pascal), you may observe a modest speedup.
Run ``nvidia-smi`` to display your GPU's architecture.



In [2]:
import torch, time, gc

# Timing utilities
start_time = None

def start_timer():
    global start_time
    gc.collect()
    torch.cuda.empty_cache()
    torch.cuda.reset_max_memory_allocated()
    torch.cuda.synchronize()
    start_time = time.time()

def end_timer_and_print(local_msg):
    torch.cuda.synchronize()
    end_time = time.time()
    print("\n" + local_msg)
    print("Total execution time = {:.3f} sec".format(end_time - start_time))
    print("Max memory used by tensors = {} bytes".format(torch.cuda.max_memory_allocated()))

A simple network
----------------
The following sequence of linear layers and ReLUs should show a speedup with mixed precision.



In [3]:
def make_model(in_size, out_size, num_layers):
    layers = []
    for _ in range(num_layers - 1):
        layers.append(torch.nn.Linear(in_size, in_size))
        layers.append(torch.nn.ReLU())
    layers.append(torch.nn.Linear(in_size, out_size))
    return torch.nn.Sequential(*tuple(layers)).cuda()

``batch_size``, ``in_size``, ``out_size``, and ``num_layers`` are chosen to be large enough to saturate the GPU with work.
Typically, mixed precision provides the greatest speedup when the GPU is saturated.
Small networks may be CPU bound, in which case mixed precision won't improve performance.
Sizes are also chosen such that linear layers' participating dimensions are multiples of 8,
to permit Tensor Core usage on Tensor Core-capable GPUs (see `Troubleshooting<troubleshooting>` below).

Exercise: Vary participating sizes and see how the mixed precision speedup changes.



In [4]:
batch_size = 512 # Try, for example, 128, 256, 513.
in_size = 4096
out_size = 4096
num_layers = 3
num_batches = 50
epochs = 3

# Creates data in default precision.
# The same data is used for both default and mixed precision trials below.
# You don't need to manually change inputs' dtype when enabling mixed precision.
data = [torch.randn(batch_size, in_size, device="cuda") for _ in range(num_batches)]
targets = [torch.randn(batch_size, out_size, device="cuda") for _ in range(num_batches)]

loss_fn = torch.nn.MSELoss().cuda()

In [5]:
data

[tensor([[ 0.4625,  1.1594, -1.8091,  ...,  0.8453,  0.6155, -1.2540],
         [ 0.5399,  0.4214,  1.3481,  ...,  1.2287, -0.1138, -1.2123],
         [ 0.5529, -0.5995,  0.2861,  ...,  0.5675, -0.2956, -0.8250],
         ...,
         [-0.6990, -0.0326,  2.1048,  ..., -0.0150,  0.7374, -2.2933],
         [-0.4058,  0.3649, -1.1202,  ...,  0.9525, -1.9334,  0.8800],
         [ 0.8994,  1.3147,  0.2422,  ...,  0.8190, -0.3359, -0.1576]],
        device='cuda:0'),
 tensor([[ 0.1332, -0.2778,  0.7885,  ..., -0.9770,  1.2424, -0.1520],
         [ 0.9859,  0.7506, -1.9478,  ...,  0.4920,  0.4249, -1.2119],
         [ 1.8029,  0.7296, -0.0392,  ...,  0.6088, -0.0899, -0.6875],
         ...,
         [ 1.2080,  1.1131, -0.3568,  ..., -1.2847, -0.4269, -0.4956],
         [-2.6009, -1.4892, -1.4090,  ..., -2.0354, -0.0121, -0.3604],
         [-1.4321,  0.4775,  1.4783,  ...,  1.0182,  0.9734,  0.1630]],
        device='cuda:0'),
 tensor([[-0.8467, -0.6810, -1.8795,  ..., -0.0405,  0.9422, -0.31

In [6]:
targets

[tensor([[ 0.7156, -1.0421,  0.6266,  ..., -1.6096,  0.8201,  2.2344],
         [ 0.4541, -0.1076, -1.1849,  ..., -0.5843,  0.7747,  0.4335],
         [ 1.1672,  0.2443,  1.7625,  ..., -1.0886, -0.8783,  0.0426],
         ...,
         [ 1.3608,  1.6578, -0.4955,  ..., -0.5900,  0.0832,  1.4340],
         [-1.2295,  2.0904, -1.5459,  ..., -1.5563, -0.8400, -1.9047],
         [-1.4365,  0.2769, -0.8063,  ...,  1.2359, -0.6476, -1.7906]],
        device='cuda:0'),
 tensor([[ 1.8659, -0.6101, -0.4861,  ...,  0.1126, -0.9398,  0.6899],
         [-0.1141, -0.6459,  1.1626,  ...,  2.0315,  0.5255, -1.1865],
         [ 0.3333, -1.5983, -1.8233,  ...,  1.0828, -0.0234,  0.1296],
         ...,
         [-1.2137, -0.5294, -0.8974,  ..., -0.3978, -2.0166,  1.0965],
         [ 0.3867, -1.2510, -0.1558,  ..., -0.4065, -0.5282,  0.1493],
         [ 0.9304,  0.8951,  1.9249,  ...,  0.9115,  1.0710, -1.9439]],
        device='cuda:0'),
 tensor([[ 0.2759,  0.8359, -1.9518,  ...,  1.0997, -0.3114,  0.65

In [8]:
len(data)

50

In [10]:
len(targets)

50

Default Precision
-----------------
Without ``torch.cuda.amp``, the following simple network executes all ops in default precision (``torch.float32``):



In [11]:
net = make_model(in_size, out_size, num_layers)
opt = torch.optim.SGD(net.parameters(), lr=0.001)

start_timer()
for epoch in range(epochs):
    for input, target in zip(data, targets):
        output = net(input)
        loss = loss_fn(output, target)
        loss.backward()
        opt.step()
        opt.zero_grad() # set_to_none=True here can modestly improve performance
end_timer_and_print("Default precision:")

/home/ubuntu/anaconda3/envs/pytorch_latest_p37/lib/python3.7/site-packages/torch/cuda/memory.py:234: FutureWarning: torch.cuda.reset_max_memory_allocated now calls torch.cuda.reset_peak_memory_stats, which resets /all/ peak memory stats.
  FutureWarning)



Default precision:
Total execution time = 1.924 sec
Max memory used by tensors = 1367458816 bytes


In [12]:
net

Sequential(
  (0): Linear(in_features=4096, out_features=4096, bias=True)
  (1): ReLU()
  (2): Linear(in_features=4096, out_features=4096, bias=True)
  (3): ReLU()
  (4): Linear(in_features=4096, out_features=4096, bias=True)
)

Adding autocast
---------------
Instances of `torch.cuda.amp.autocast <https://pytorch.org/docs/stable/amp.html#autocasting>`_
serve as context managers that allow regions of your script to run in mixed precision.

In these regions, CUDA ops run in a dtype chosen by autocast
to improve performance while maintaining accuracy.
See the `Autocast Op Reference <https://pytorch.org/docs/stable/amp.html#autocast-op-reference>`_
for details on what precision autocast chooses for each op, and under what circumstances.



In [13]:
for epoch in range(0): # 0 epochs, this section is for illustration only
    for input, target in zip(data, targets):
        # Runs the forward pass under autocast.
        with torch.cuda.amp.autocast():
            output = net(input)
            # output is float16 because linear layers autocast to float16.
            assert output.dtype is torch.float16

            loss = loss_fn(output, target)
            # loss is float32 because mse_loss layers autocast to float32.
            assert loss.dtype is torch.float32

        # Exits autocast before backward().
        # Backward passes under autocast are not recommended.
        # Backward ops run in the same dtype autocast chose for corresponding forward ops.
        loss.backward()
        opt.step()
        opt.zero_grad() # set_to_none=True here can modestly improve performance

Adding GradScaler
-----------------
`Gradient scaling <https://pytorch.org/docs/stable/amp.html#gradient-scaling>`_
helps prevent gradients with small magnitudes from flushing to zero
("underflowing") when training with mixed precision.

`torch.cuda.amp.GradScaler <https://pytorch.org/docs/stable/amp.html#torch.cuda.amp.GradScaler>`_
performs the steps of gradient scaling conveniently.



In [14]:
# Constructs scaler once, at the beginning of the convergence run, using default args.
# If your network fails to converge with default GradScaler args, please file an issue.
# The same GradScaler instance should be used for the entire convergence run.
# If you perform multiple convergence runs in the same script, each run should use
# a dedicated fresh GradScaler instance.  GradScaler instances are lightweight.
scaler = torch.cuda.amp.GradScaler()

for epoch in range(0): # 0 epochs, this section is for illustration only
    for input, target in zip(data, targets):
        with torch.cuda.amp.autocast():
            output = net(input)
            loss = loss_fn(output, target)

        # Scales loss.  Calls backward() on scaled loss to create scaled gradients.
        scaler.scale(loss).backward()

        # scaler.step() first unscales the gradients of the optimizer's assigned params.
        # If these gradients do not contain infs or NaNs, optimizer.step() is then called,
        # otherwise, optimizer.step() is skipped.
        scaler.step(opt)

        # Updates the scale for next iteration.
        scaler.update()

        opt.zero_grad() # set_to_none=True here can modestly improve performance

All together: "Automatic Mixed Precision"
------------------------------------------
(The following also demonstrates ``enabled``, an optional convenience argument to ``autocast`` and ``GradScaler``.
If False, ``autocast`` and ``GradScaler``\ 's calls become no-ops.
This allows switching between default precision and mixed precision without if/else statements.)



In [17]:
use_amp = True

net = make_model(in_size, out_size, num_layers)
opt = torch.optim.SGD(net.parameters(), lr=0.001)
scaler = torch.cuda.amp.GradScaler(enabled=use_amp)

start_timer()
for epoch in range(epochs):
    for input, target in zip(data, targets):
        with torch.cuda.amp.autocast(enabled=use_amp):
            output = net(input)
            print(len(output))
            loss = loss_fn(output, target)
        scaler.scale(loss).backward()
        scaler.step(opt)
        scaler.update()
        opt.zero_grad() # set_to_none=True here can modestly improve performance
end_timer_and_print("Mixed precision:")

512
512
512
512
512
512
512
512
512
512
512
512
512
512
512
512
512
512
512
512
512
512
512
512
512
512
512
512
512
512
512
512
512
512
512
512
512
512
512
512
512
512
512
512
512
512
512
512
512
512
512
512
512
512
512
512
512
512
512
512
512
512
512
512
512
512
512
512
512
512
512
512
512
512
512
512
512
512
512
512
512
512
512
512
512
512
512
512
512
512
512
512
512
512
512
512
512
512
512
512
512
512
512
512
512
512
512
512
512
512
512
512
512
512
512
512
512
512
512
512
512
512
512
512
512
512
512
512
512
512
512
512
512
512
512
512
512
512
512
512
512
512
512
512
512
512
512
512
512
512

Mixed precision:
Total execution time = 0.768 sec
Max memory used by tensors = 1988372992 bytes


Inspecting/modifying gradients (e.g., clipping)
--------------------------------------------------------
All gradients produced by ``scaler.scale(loss).backward()`` are scaled.  If you wish to modify or inspect
the parameters' ``.grad`` attributes between ``backward()`` and ``scaler.step(optimizer)``, you should
unscale them first using `scaler.unscale_(optimizer) <https://pytorch.org/docs/stable/amp.html#torch.cuda.amp.GradScaler.unscale_>`_.



In [ ]:
for epoch in range(0): # 0 epochs, this section is for illustration only
    for input, target in zip(data, targets):
        with torch.cuda.amp.autocast():
            output = net(input)
            loss = loss_fn(output, target)
        scaler.scale(loss).backward()

        # Unscales the gradients of optimizer's assigned params in-place
        scaler.unscale_(opt)

        # Since the gradients of optimizer's assigned params are now unscaled, clips as usual.
        # You may use the same value for max_norm here as you would without gradient scaling.
        torch.nn.utils.clip_grad_norm_(net.parameters(), max_norm=0.1)

        scaler.step(opt)
        scaler.update()
        opt.zero_grad() # set_to_none=True here can modestly improve performance

Saving/Resuming
----------------
To save/resume Amp-enabled runs with bitwise accuracy, use
`scaler.state_dict <https://pytorch.org/docs/stable/amp.html#torch.cuda.amp.GradScaler.state_dict>`_ and
`scaler.load_state_dict <https://pytorch.org/docs/stable/amp.html#torch.cuda.amp.GradScaler.load_state_dict>`_.

When saving, save the scaler state dict alongside the usual model and optimizer state dicts.
Do this either at the beginning of an iteration before any forward passes, or at the end of
an iteration after ``scaler.update()``.



In [ ]:
checkpoint = {"model": net.state_dict(),
              "optimizer": opt.state_dict(),
              "scaler": scaler.state_dict()}
# Write checkpoint as desired, e.g.,
# torch.save(checkpoint, "filename")

When resuming, load the scaler state dict alongside the model and optimizer state dicts.



In [ ]:
# Read checkpoint as desired, e.g.,
# dev = torch.cuda.current_device()
# checkpoint = torch.load("filename",
#                         map_location = lambda storage, loc: storage.cuda(dev))
net.load_state_dict(checkpoint["model"])
opt.load_state_dict(checkpoint["optimizer"])
scaler.load_state_dict(checkpoint["scaler"])

If a checkpoint was created from a run *without* Amp, and you want to resume training *with* Amp,
load model and optimizer states from the checkpoint as usual.  The checkpoint won't contain a saved scaler state, so
use a fresh instance of ``GradScaler``.

If a checkpoint was created from a run *with* Amp and you want to resume training *without* Amp,
load model and optimizer states from the checkpoint as usual, and ignore the saved scaler state.



Inference/Evaluation
--------------------
``autocast`` may be used by itself to wrap inference or evaluation forward passes. ``GradScaler`` is not necessary.




Advanced topics
---------------
See the `Automatic Mixed Precision Examples <https://pytorch.org/docs/stable/notes/amp_examples.html>`_ for advanced use cases including:

* Gradient accumulation
* Gradient penalty/double backward
* Networks with multiple models, optimizers, or losses
* Multiple GPUs (``torch.nn.DataParallel`` or ``torch.nn.parallel.DistributedDataParallel``)
* Custom autograd functions (subclasses of ``torch.autograd.Function``)

If you perform multiple convergence runs in the same script, each run should use
a dedicated fresh GradScaler instance.  GradScaler instances are lightweight.

If you're registering a custom C++ op with the dispatcher, see the
`autocast section <https://pytorch.org/tutorials/advanced/dispatcher.html#autocast>`_
of the dispatcher tutorial.




Troubleshooting
---------------
Speedup with Amp is minor
~~~~~~~~~~~~~~~~~~~~~~~~~
1. Your network may fail to saturate the GPU(s) with work, and is therefore CPU bound. Amp's effect on GPU performance
   won't matter.

   * A rough rule of thumb to saturate the GPU is to increase batch and/or network size(s)
     as much as you can without running OOM.
   * Try to avoid excessive CPU-GPU synchronization (``.item()`` calls, or printing values from CUDA tensors).
   * Try to avoid sequences of many small CUDA ops (coalesce these into a few large CUDA ops if you can).
2. Your network may be GPU compute bound (lots of matmuls/convolutions) but your GPU does not have Tensor Cores.
   In this case a reduced speedup is expected.
3. Matmul dimensions are not Tensor Core-friendly.  Make sure matmuls' participating sizes are multiples of 8.
   (For NLP models with encoders/decoders, this can be subtle.  Also, convolutions used to have similar size constraints
   for Tensor Core use, but for CuDNN versions 7.3 and later, no such constraints exist.  See
   `here <https://github.com/NVIDIA/apex/issues/221#issuecomment-478084841>`_ for guidance.)

Loss is inf/NaN
~~~~~~~~~~~~~~~
First, check if your network fits an `advanced use case<advanced-topics>`.
See also `Prefer binary_cross_entropy_with_logits over binary_cross_entropy <https://pytorch.org/docs/stable/amp.html#prefer-binary-cross-entropy-with-logits-over-binary-cross-entropy>`_.

If you're confident your Amp usage is correct, you may need to file an issue, but before doing so, it's helpful to gather the following information:

1. Disable ``autocast`` or ``GradScaler`` individually (by passing ``enabled=False`` to their constructor) and see if infs/NaNs persist.
2. If you suspect part of your network (e.g., a complicated loss function) overflows , run that forward region in ``float32``
   and see if infs/NaNs persist.
   `The autocast docstring <https://pytorch.org/docs/stable/amp.html#torch.cuda.amp.autocast>`_'s last code snippet
   shows forcing a subregion to run in ``float32`` (by locally disabling autocast and casting the subregion's inputs).

Type mismatch error (may manifest as CUDNN_STATUS_BAD_PARAM)
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Autocast tries to cover all ops that benefit from or require casting.
`Ops that receive explicit coverage <https://pytorch.org/docs/stable/amp.html#autocast-op-reference>`_
are chosen based on numerical properties, but also on experience.
If you see a type mismatch error in an autocast-enabled forward region or a backward pass following that region,
it's possible autocast missed an op.

Please file an issue with the error backtrace.  ``export TORCH_SHOW_CPP_STACKTRACES=1`` before running your script to provide
fine-grained information on which backend op is failing.

